# ABS Monthly Lending 5601

## Python set-up

In [1]:
# analytic imports
import matplotlib.pyplot as plt
import pandas as pd

# local imports
from abs_data_capture import (
    get_abs_meta_and_data,
    get_fs_constants,
    get_meta_constants,
    get_plot_constants,
)
from plotting import (
    clear_chart_dir,
    line_plot,
    recalibrate,
    seas_trend_plot,
    set_chart_dir,
)

# pandas display settings
pd.options.display.max_rows = 9999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# display charts in this notebook
SHOW = False

## Get data from ABS

In [2]:
CATALOGUE_ID = "5601"
source, CHART_DIR, META_DATA = get_fs_constants(CATALOGUE_ID)

# The ABS has the data in multiple zip files, therefore
# this is a bit more complicated than what I usually do
abs_dict = {}
for url_num in [0, 1, 2, 3, 4, 5]:
    # get zip file and tables from ABS
    d = get_abs_meta_and_data(CATALOGUE_ID, table=url_num, verbose=False)

    # handle meta data
    m = d[META_DATA]
    if META_DATA not in abs_dict:
        abs_dict[META_DATA] = m
    else:
        abs_dict[META_DATA] = pd.concat([abs_dict[META_DATA], m])
    del d[META_DATA]

    # handle xlsx files
    abs_dict = {**abs_dict, **d}

meta = abs_dict[META_DATA]
RECENT, plot_times, plot_tags = get_plot_constants(meta)
did_col, id_col, table_col, type_col, unit_col, tdec_col = get_meta_constants()

Found URL for a ZIP file on ABS web page
Retrieving data from the cache file: ABS_CACHE/16134a2a02889ad6103e2c6b5a67dea9--Housing-Finance-Total.zip
Extracting DataFrames from the zip-file ...
Found URL for a ZIP file on ABS web page
Retrieving data from the cache file: ABS_CACHE/d721e362efc627396c728ec8e4dd10af--Housing-Finance-Owner-occupiers.zip
Extracting DataFrames from the zip-file ...
Found URL for a ZIP file on ABS web page
Retrieving data from the cache file: ABS_CACHE/7a544f829dc29ef51b14de3aa368c1ef--Housing-Finance-Investors.zip
Extracting DataFrames from the zip-file ...
Found URL for a ZIP file on ABS web page
Retrieving data from the cache file: ABS_CACHE/514308a4092240f2a869dc6aeb11f7c2--Housing-Finance-First-home-buyers.zip
Extracting DataFrames from the zip-file ...
Found URL for a ZIP file on ABS web page
Retrieving data from the cache file: ABS_CACHE/ffae557b48bcec4cf046c14e1780084a--Personal-Finance.zip
Extracting DataFrames from the zip-file ...
Found URL for a ZIP

## Plotting

### Set-up

In [3]:
def chart_settings(chart_dir: str = CHART_DIR) -> None:
    """Set up charting parameters."""

    set_chart_dir(chart_dir)
    clear_chart_dir(chart_dir)
    plt.style.use("fivethirtyeight")


chart_settings()

### Headline Indicators

In [4]:
def fix_title(title, lfooter):
    """Title beautification."""

    check = [
        "Total housing excluding refinancing",
        "Total purpose excluding refinancing",
        "New loan commitments",
        "Fixed term loans",
        "Value",
    ]
    for c in check:
        text = f"{c} ;"
        if text in title:
            title = title.replace(text, "")
            lfooter = lfooter + f"{c}. "
    title = (
        title.replace("Businesses", "")
        .replace("Business Finance", "Business Finance -")
        .replace("Households", "Households -")
        .replace(";", "")
        .replace("    ", " ")
        .replace("   ", " ")
        .replace("  ", " ")
        .strip()
    )
    return title, lfooter

In [5]:
def headline_charts() -> None:
    """Produce headline charts."""

    rows = [
        "Households ;  Housing Finance ;  Total housing excluding refinancing ;  New loan commitments ;",
        "Businesses ;  Business Finance ;  Fixed term loans ;  Purchase of property ;  New loan commitments ;",
        "Businesses ;  Business Finance ;  Fixed term loans ;  Construction ;  New loan commitments ;  Value ;",
        "Households ;  Personal Finance ;  Fixed term loans ;  Total purpose excluding refinancing ;  New loan commitments ;",
    ]

    for headline in rows:
        # get data
        f = pd.DataFrame()
        for series_type in "Seasonally Adjusted", "Trend":
            found = meta[
                meta[type_col].str.contains(series_type)
                & meta[did_col].str.contains(headline)
            ]
            assert len(found) == 1
            row = found.iloc[0]
            series_id, units, table, did = (
                row[id_col],
                row[unit_col],
                row[table_col],
                row[did_col],
            )
            f[series_type] = (abs_dict[table])[series_id]
        f_, units = recalibrate(f, units)

        # plot
        title, lfooter = fix_title(did, "Australia. ")
        seas_trend_plot(
            pd.DataFrame(f_),  # mypy
            starts=plot_times,
            tags=plot_tags,
            title=title,
            ylabel=f"{units}",
            lfooter=lfooter,
            rfooter=f"{source} T{table}",
            show=SHOW,
        )


headline_charts()

### Business

In [6]:
def business_charts() -> None:
    """Produce business charts."""

    rows = meta[
        meta[did_col].str.contains("Business")
        & meta[did_col].str.contains("loans")
        & meta[did_col].str.contains("Total purpose")
    ]
    for _, row in rows.iterrows():
        series_id, units, table, did, series_type = (
            row[id_col],
            row[unit_col],
            row[table_col],
            row[did_col],
            row[type_col],
        )
        series, units = recalibrate(abs_dict[table][series_id], units)
        title, lfooter = fix_title(did, f"Australia. {series_type} series. ")
        title = title.replace("Businesses", "").strip()
        line_plot(
            series,
            title=title,
            ylabel=f"{units}",
            lfooter=lfooter,
            rfooter=f"{source} T{table}",
            show=SHOW,
        )


business_charts()

## Finished

In [7]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Sat Dec 30 2023 14:37:12

Python implementation: CPython
Python version       : 3.12.1
IPython version      : 8.19.0

pandas    : 2.1.4
matplotlib: 3.8.2

Watermark: 2.4.3



In [8]:
print("Done")

Done
